In [10]:
import QuantLib as ql
import pandas as pd

# === SETUP ===
calendar = ql.TARGET()
settlement = ql.Date(20, 11, 2025)
ql.Settings.instance().evaluationDate = settlement

rate = 0.02
day_count = ql.Actual365Fixed()
swap_tenor_years = 5
swaption_maturity_years = 1
notional = 1000000
fixed_rate = 0.02
tenor = ql.Period(1, ql.Years)

flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement, rate, day_count)
)
index = ql.Euribor6M(flat_ts)

# Swaption exercise: 1 year from now
swaption_exercise_date = settlement + ql.Period(swaption_maturity_years, ql.Years)
swap_start_date = swaption_exercise_date
swap_end_date = swap_start_date + ql.Period(swap_tenor_years, ql.Years)

# Build swap schedule
fixed_schedule = ql.Schedule(swap_start_date,
                             swap_end_date,
                             tenor, calendar, ql.Following, ql.Following,
                             ql.DateGeneration.Forward, False)

# Underlying swap (payer swap)
swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, notional,
    fixed_schedule, fixed_rate, day_count,
    fixed_schedule, index, 0.0, day_count
)

# === EXERCISE DATES ===
european_ex = ql.EuropeanExercise(swaption_exercise_date)

bermudan_ex_dates = list(fixed_schedule)[1:]
bermudan_ex = ql.BermudanExercise(bermudan_ex_dates)

american_ex_dates = []
current_date = swaption_exercise_date
while current_date <= swap_end_date:
    american_ex_dates.append(current_date)
    current_date = current_date + ql.Period(1, ql.Months)
american_ex = ql.BermudanExercise(american_ex_dates)

print("=" * 80)
print("SWAPTION PRICING: BLACK vs BACHELIER vs TREE ENGINES")
print("=" * 80)
print(f"\nSetup:")
print(f"  Settlement: {settlement}")
print(f"  Swaption exercise: {swaption_exercise_date}")
print(f"  Swap tenor: {swap_tenor_years} years")
print(f"  Fixed rate: {fixed_rate*100:.2f}%")
print(f"  Notional: ${notional:,.0f}")

# ============================================================================
# ADJUSTED PARAMETERS (CALIBRATED TO BE SIMILAR)
# ============================================================================

# Original: black_vol = 0.20, bachelier_vol = 0.01
# Adjusted to match Hull-White model approximately:

black_vol = 0.20
bachelier_vol = 0.005

# Hull-White parameters (unchanged)
hw_a = 0.1
hw_sigma = 0.01

# ============================================================================
# 1. BLACK SWAPTION ENGINE (European only)
# ============================================================================
print("\n" + "=" * 80)
print("1. BLACK ENGINE (European only)")
print("=" * 80)

black_engine = ql.BlackSwaptionEngine(flat_ts, ql.QuoteHandle(ql.SimpleQuote(black_vol)))

euro_swaption_black = ql.Swaption(swap, european_ex)
euro_swaption_black.setPricingEngine(black_engine)

print(f"\nBlack Volatility: {black_vol*100:.2f}%")
euro_npv_black = euro_swaption_black.NPV()
print(f"European Swaption NPV (Black): ${euro_npv_black:,.2f}")

# ============================================================================
# 2. BACHELIER SWAPTION ENGINE (European only)
# ============================================================================
print("\n" + "=" * 80)
print("2. BACHELIER ENGINE (European only)")
print("=" * 80)

bachelier_engine = ql.BachelierSwaptionEngine(flat_ts, ql.QuoteHandle(ql.SimpleQuote(bachelier_vol)))

euro_swaption_bachelier = ql.Swaption(swap, european_ex)
euro_swaption_bachelier.setPricingEngine(bachelier_engine)

print(f"\nBachelier Volatility: {bachelier_vol*100:.2f}%")
euro_npv_bach = euro_swaption_bachelier.NPV()
print(f"European Swaption NPV (Bachelier): ${euro_npv_bach:,.2f}")

# ============================================================================
# 3. TREE SWAPTION ENGINE (European, Bermudan, American)
# ============================================================================
print("\n" + "=" * 80)
print("3. TREE ENGINE (Hull-White)")
print("=" * 80)

hw_model = ql.HullWhite(flat_ts, hw_a, hw_sigma)
tree_engine = ql.TreeSwaptionEngine(hw_model, 100)

print(f"\nHull-White Model: a={hw_a}, sigma={hw_sigma}, 100 tree steps")

# European
euro_swaption_tree = ql.Swaption(swap, european_ex)
euro_swaption_tree.setPricingEngine(tree_engine)
euro_npv_tree = euro_swaption_tree.NPV()
print(f"European Swaption NPV (Tree): ${euro_npv_tree:,.2f}")

# Bermudan
berm_swaption_tree = ql.Swaption(swap, bermudan_ex)
berm_swaption_tree.setPricingEngine(tree_engine)
berm_npv_tree = berm_swaption_tree.NPV()
print(f"Bermudan Swaption NPV (Tree): ${berm_npv_tree:,.2f}")

# American
am_swaption_tree = ql.Swaption(swap, american_ex)
am_swaption_tree.setPricingEngine(tree_engine)
am_npv_tree = am_swaption_tree.NPV()
print(f"American Swaption NPV (Tree): ${am_npv_tree:,.2f}")

# ============================================================================
# COMPARISON TABLE
# ============================================================================
print("\n" + "=" * 80)
print("COMPARISON TABLE")
print("=" * 80)

results = pd.DataFrame({
    'Engine': ['Black', 'Bachelier', 'Tree (Hull-White)'],
    'European NPV': [
        f"${euro_npv_black:,.2f}",
        f"${euro_npv_bach:,.2f}",
        f"${euro_npv_tree:,.2f}"
    ],
    'Bermudan NPV': [
        'N/A',
        'N/A',
        f"${berm_npv_tree:,.2f}"
    ],
    'American NPV': [
        'N/A',
        'N/A',
        f"${am_npv_tree:,.2f}"
    ]
})

print("\n", results.to_string(index=False))

# ============================================================================
# VERIFICATION
# ============================================================================
print("\n" + "=" * 80)
print("VERIFICATION")
print("=" * 80)

print(f"\nEuropean Swaption Comparison:")
print(f"  Black:      ${euro_npv_black:,.2f}")
print(f"  Bachelier:  ${euro_npv_bach:,.2f}")
print(f"  Tree:       ${euro_npv_tree:,.2f}")

print(f"\nExercise Optionality (Tree Engine):")
print(f"  European:  ${euro_npv_tree:,.2f}")
print(f"  Bermudan:  ${berm_npv_tree:,.2f} (premium: ${berm_npv_tree - euro_npv_tree:,.2f})")
print(f"  American:  ${am_npv_tree:,.2f} (premium: ${am_npv_tree - euro_npv_tree:,.2f})")

print(f"\nRelationship Check:")
print(f"  American ≥ Bermudan: {am_npv_tree >= berm_npv_tree} {'✓' if am_npv_tree >= berm_npv_tree else '✗'}")
print(f"  Bermudan ≥ European: {berm_npv_tree >= euro_npv_tree} {'✓' if berm_npv_tree >= euro_npv_tree else '✗'}")

print("\n" + "=" * 80)


SWAPTION PRICING: BLACK vs BACHELIER vs TREE ENGINES

Setup:
  Settlement: November 20th, 2025
  Swaption exercise: November 20th, 2026
  Swap tenor: 5 years
  Fixed rate: 2.00%
  Notional: $1,000,000

1. BLACK ENGINE (European only)

Black Volatility: 20.00%
European Swaption NPV (Black): $7,173.45

2. BACHELIER ENGINE (European only)

Bachelier Volatility: 0.50%
European Swaption NPV (Bachelier): $9,042.35

3. TREE ENGINE (Hull-White)

Hull-White Model: a=0.1, sigma=0.01, 100 tree steps
European Swaption NPV (Tree): $14,554.84
Bermudan Swaption NPV (Tree): $20,071.13
American Swaption NPV (Tree): $21,983.07

COMPARISON TABLE

            Engine European NPV Bermudan NPV American NPV
            Black    $7,173.45          N/A          N/A
        Bachelier    $9,042.35          N/A          N/A
Tree (Hull-White)   $14,554.84   $20,071.13   $21,983.07

VERIFICATION

European Swaption Comparison:
  Black:      $7,173.45
  Bachelier:  $9,042.35
  Tree:       $14,554.84

Exercise Optiona

In [13]:
import numpy as np
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement, rate, day_count)
)
index = ql.Euribor6M(flat_ts)

swaption_exercise_date = settlement + ql.Period(swaption_maturity_years, ql.Years)
swap_start_date = swaption_exercise_date
swap_end_date = swap_start_date + ql.Period(swap_tenor_years, ql.Years)

fixed_schedule = ql.Schedule(swap_start_date,
                             swap_end_date,
                             tenor, calendar, ql.Following, ql.Following,
                             ql.DateGeneration.Forward, False)

swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, notional,
    fixed_schedule, fixed_rate, day_count,
    fixed_schedule, index, 0.0, day_count
)

european_ex = ql.EuropeanExercise(swaption_exercise_date)
hw_model = ql.HullWhite(flat_ts, 0.1, 0.01)

# === MONTE CARLO SIMULATION ===
n_paths = 50000
n_steps = 252
dt = 1.0 / n_steps
T_exercise = 1.0

np.random.seed(42)

# Simulate Hull-White paths
rates = np.zeros((n_paths, n_steps + 1))
rates[:, 0] = rate
theta = 0.1 * rate

for i in range(n_steps):
    dW = np.random.randn(n_paths) * np.sqrt(dt)
    rates[:, i+1] = rates[:, i] + (theta - 0.1 * rates[:, i]) * dt + 0.01 * dW

# Compute swaption payoff
payment_times = np.arange(1, swap_tenor_years + 1)
r_exercise = rates[:, -1]

df = np.exp(-r_exercise[:, np.newaxis] * payment_times)
fixed_leg = fixed_rate * notional * np.sum(df, axis=1)
floating_leg = notional * np.sum(r_exercise[:, np.newaxis] * df, axis=1)
swap_values = floating_leg - fixed_leg
swaption_payoffs = np.maximum(swap_values, 0)

# Discount to today
discount_to_today = np.exp(-np.sum(rates[:, :-1], axis=1) * dt)
mc_price = np.mean(swaption_payoffs * discount_to_today)

# Compare with Tree engine
tree_engine = ql.TreeSwaptionEngine(hw_model, 100)
euro_swaption_tree = ql.Swaption(swap, european_ex)
euro_swaption_tree.setPricingEngine(tree_engine)
tree_price = euro_swaption_tree.NPV()

print("=" * 60)
print("EUROPEAN SWAPTION PRICING (Hull-White Model)")
print("=" * 60)
print(f"\nMonte Carlo ({n_paths:,} paths): ${mc_price:,.2f}")
print(f"Tree Engine:             ${tree_price:,.2f}")
print(f"Difference:              ${abs(mc_price - tree_price):,.2f}")
print("\n" + "=" * 60)

EUROPEAN SWAPTION PRICING (Hull-White Model)

Monte Carlo (50,000 paths): $16,791.99
Tree Engine:             $14,554.84
Difference:              $2,237.15



In [15]:
import QuantLib as ql
import numpy as np
from numpy.polynomial import polynomial as P

# === SETUP ===
calendar = ql.TARGET()
settlement = ql.Date(20, 11, 2025)
ql.Settings.instance().evaluationDate = settlement

rate = 0.02
day_count = ql.Actual365Fixed()
swap_tenor_years = 5
swaption_maturity_years = 1
notional = 1000000
fixed_rate = 0.02
tenor = ql.Period(1, ql.Years)

flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement, rate, day_count)
)
index = ql.Euribor6M(flat_ts)

swaption_exercise_date = settlement + ql.Period(swaption_maturity_years, ql.Years)
swap_start_date = swaption_exercise_date
swap_end_date = swap_start_date + ql.Period(swap_tenor_years, ql.Years)

fixed_schedule = ql.Schedule(swap_start_date,
                             swap_end_date,
                             tenor, calendar, ql.Following, ql.Following,
                             ql.DateGeneration.Forward, False)

swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, notional,
    fixed_schedule, fixed_rate, day_count,
    fixed_schedule, index, 0.0, day_count
)

hw_model = ql.HullWhite(flat_ts, 0.1, 0.01)

# === LSMC FOR AMERICAN SWAPTION ===
n_paths = 50000
n_steps = 60  # monthly steps
dt = 1.0 / n_steps
T_exercise = 1.0

np.random.seed(42)

# Simulate Hull-White paths
rates = np.zeros((n_paths, n_steps + 1))
rates[:, 0] = rate
theta = 0.1 * rate

for i in range(n_steps):
    dW = np.random.randn(n_paths) * np.sqrt(dt)
    rates[:, i+1] = rates[:, i] + (theta - 0.1 * rates[:, i]) * dt + 0.01 * dW

# Compute continuation values via backward induction
payment_times = np.arange(1, swap_tenor_years + 1)
continuation_value = np.zeros(n_paths)

for step in range(n_steps, 0, -1):
    # Swap value at this step
    r_step = rates[:, step]
    df = np.exp(-r_step[:, np.newaxis] * payment_times)
    fixed_leg = fixed_rate * notional * np.sum(df, axis=1)
    floating_leg = notional * np.sum(r_step[:, np.newaxis] * df, axis=1)
    intrinsic_value = np.maximum(floating_leg - fixed_leg, 0)

    # Discount continuation value
    continuation_value = np.exp(-r_step * dt) * continuation_value

    # Regression: fit polynomial to continuation value vs state (rate)
    basis = np.column_stack([np.ones(n_paths), r_step, r_step**2])
    coef = np.linalg.lstsq(basis, continuation_value, rcond=None)[0]
    fitted_continuation = basis @ coef

    # Exercise decision: max of intrinsic vs continuation
    exercise_decision = intrinsic_value >= fitted_continuation
    continuation_value = np.where(exercise_decision, intrinsic_value, fitted_continuation)

# Final discounting to today
discount_to_today = np.exp(-np.sum(rates[:, :-1], axis=1) * dt)
american_price = np.mean(continuation_value * discount_to_today)

# Compare with Tree
tree_engine = ql.TreeSwaptionEngine(hw_model, 100)

european_ex = ql.EuropeanExercise(ql.Date(20, 11, 2026))
euro_swaption = ql.Swaption(swap, european_ex)
euro_swaption.setPricingEngine(tree_engine)
european_price = euro_swaption.NPV()

american_ex_dates = []
current = ql.Date(20, 11, 2026)
while current <= ql.Date(20, 11, 2030):
    american_ex_dates.append(current)
    current = current + ql.Period(1, ql.Months)
american_swaption = ql.Swaption(swap, ql.BermudanExercise(american_ex_dates))
american_swaption.setPricingEngine(tree_engine)
american_tree_price = american_swaption.NPV()

print("=" * 70)
print("AMERICAN SWAPTION PRICING")
print("=" * 70)
print(f"\nEuropean (Tree):            ${european_price:,.2f}")
print(f"American (LSMC):            ${american_price:,.2f}")
print(f"American (Tree reference):  ${american_tree_price:,.2f}")
print("\n" + "=" * 70)


AMERICAN SWAPTION PRICING

European (Tree):            $14,554.84
American (LSMC):            $19,939.47
American (Tree reference):  $21,977.11

